La red neuronal que armamos no cuenta con capas intermedias/ocultas. Esta la de entrada, que es un dataset descargado de Kaggle, y la salida, que intenta predecir el tipo de conductor(por ejemplo, casual)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Creamos la funcion de activación sigmoide, ideal para clasificación binaria ya que devuelve valores entre 1 y 0, que se puede interpretar como que tan probable es que un conjunto de muetra pertenezca a una clase.
# Esta funcion de activación es la que se encontrará en la neurona de salida
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [3]:
#declaramos dos tipos de funciones diferentes para calcular el error. La linear, que resta las predicciones a los valores reales, y la mean absolute, que calcula el error e ignora en que dirección
# fue este error(si es negativo o positivo)
def linear_error(y_true, y_pred):
    return np.mean(y_true - y_pred)

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [4]:
#Funcion que randomiza los weights y los bias iniciales
def initialize_weights(input_size, output_size):
    weights = np.random.randn(input_size, output_size) * 0.01
    bias = np.zeros((1, output_size))
    return weights, bias

In [5]:
#Funcion que hace la propagacion hacia adelante, myultiplicando las X por sus weights y luego sumando el bias. Y al final, pasando el resultado por la funcion de activacion sigmoide
def forward_pass(X, weights, bias):
    z = np.dot(X, weights) + bias
    return sigmoid(z)

In [6]:
#Creamos la funcion de propagacion hacia atras para bque pueda ajustar los weights y bias aprendiendo de su error
def backpropagation(X, y, output, weights, bias, learning_rate):
    # Calculamos el error entre la salida esperada y la salida real (output)
    # `error` representa cuán lejos está la predicción de la verdad
    error = output - y
    
    # Calculamos el gradiente de los weights
    # `sigmoid_derivative(output)` calcula la derivada de la función de activación 
    # `X.T` es la transposición de la matriz de entrada, lo que permite realizar el producto punto correctamente
    d_weights = np.dot(X.T, error * sigmoid_derivative(output))

    # Calculamos el gradiente del bias
    # Sumamos el error ajustado por la derivada de la función de activación a lo largo de las filas
    d_bias = np.sum(error * sigmoid_derivative(output), axis=0, keepdims=True)
    
    # Ajustamos los weights y bias usando la tasa de aprendizaje
    # `learning_rate` controla cuánto se ajustan los pesos en cada iteración
    weights -= learning_rate * d_weights
    bias -= learning_rate * d_bias

    return weights, bias

In [7]:
#Creamos la funcion que entrenara a la NN
def train(X, y, input_size, output_size, epochs, learning_rate):
    # Usamos la funcion previamente crada oara crear pesos y bias
    weights, bias = initialize_weights(input_size, output_size)
    
    # Entrenamos por varias epochs (veces que se hará todo el proceso de ir para delante, para atras y actualizar weights y bias)
    for epoch in range(epochs):
        # Propagación hacia adelante
        output = forward_pass(X, weights, bias)
        
        # Calculo de pérdida
        loss = linear_error(y, output)
        
        # Retropropagación
        weights, bias = backpropagation(X, y, output, weights, bias, learning_rate)
        
        # Imprime el progreso cada 100 épocas para chequear que tan bien o mal va el aprendizaje
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    return weights, bias

In [8]:
# Se cra la funcion con la que se va a predecir 
def predict(X, weights, bias):
    output = forward_pass(X, weights, bias)
    # return (output > 0.3).astype(int)
    return output

In [9]:
# #x = ([precio,metroscuadrados,pisomarmol,pisomadera,hornoagas,hornoelectrico])
# #y = ([moderna,rustica,barata])

# X = np.array([
#     [150000, 120, 1, 0, 1, 0],
#     [80000, 80, 0, 1, 0, 1],
#     [200000, 150, 1, 1, 1, 1],
#     [50000, 60, 0, 0, 0, 1],
#     [120000, 100, 1, 0, 1, 0]
# ])


# y = np.array([
#     [1, 0, 0],
#     [0, 1, 1],
#     [1, 0, 0],
#     [0, 1, 1],
#     [1, 0, 0] 
# ])


La red tiene X cantidad de entradas (desconzco la cantidad de filas). Cada entrada(fila en el caso del primer modelo) va a tener su respectivo weight y va a haber un bias para la capa.
En el caso del modelo con capas ocultas, cada entrada (tanto las del dataset como las creadas por el modelo) va a tener su respectivo weight y va a haber un bias por cada neurona de salida de la capa.

In [10]:
##https://www.kaggle.com/datasets/valakhorasani/electric-vehicle-charging-patterns

#Importamos un dataset y sacamos filas con valores null y ciertas columnas que consideramos innecearias
dataset = pd.read_csv("ev_charging_patterns.csv")
dataset = dataset.dropna()
dataset = dataset.drop(columns=['User ID', 'Charging Station ID', 'Charging Station Location','Charging Start Time','Charging End Time','Time of Day','Day of Week','Charger Type'])
dataset.head(1)

,Vehicle Model,Battery Capacity (kWh),Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),User Type
0,BMW i3,108.463007,60.712346,0.591363,36.389181,13.087717,29.371576,86.119962,293.602111,27.947953,2.0,Commuter


In [11]:
#Nos guardamos todas las columnas excepto la que intentamos predecir en una variable X
X = dataset.drop('User Type', axis=1)
X = pd.get_dummies(X, drop_first=False).astype(int).to_numpy()

#Guardamos la variable a predecir en una llamada Y
y = dataset['User Type']
y= pd.get_dummies(y, drop_first=False).astype(int).to_numpy()
y

#Ambas las tranformamos a matrices de numpy para poder manipularlas con las funciones de arriba y las que siguen abajo

array([[0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [12]:
#Fiteamos X e Y para minimizar el error
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(y)

In [13]:
# Ponemos los hiperparametros
input_size = X.shape[1]
output_size = y.shape[1]
epochs = 3000
learning_rate = 0.01

In [14]:
# Entrenamos al modelo

weights, bias = train(X, y, input_size, output_size, epochs, learning_rate)

Epoch 0, Loss: -0.5000
Epoch 100, Loss: -0.0599
Epoch 200, Loss: -0.0539
Epoch 300, Loss: -0.0514
Epoch 400, Loss: -0.0499
Epoch 500, Loss: -0.0492
Epoch 600, Loss: -0.0488
Epoch 700, Loss: -0.0484
Epoch 800, Loss: -0.0481
Epoch 900, Loss: -0.0480
Epoch 1000, Loss: -0.0484
Epoch 1100, Loss: -0.0490
Epoch 1200, Loss: -0.0495
Epoch 1300, Loss: -0.0496
Epoch 1400, Loss: -0.0497
Epoch 1500, Loss: -0.0496
Epoch 1600, Loss: -0.0496
Epoch 1700, Loss: -0.0496
Epoch 1800, Loss: -0.0495
Epoch 1900, Loss: -0.0495
Epoch 2000, Loss: -0.0495
Epoch 2100, Loss: -0.0494
Epoch 2200, Loss: -0.0494
Epoch 2300, Loss: -0.0494
Epoch 2400, Loss: -0.0494
Epoch 2500, Loss: -0.0494
Epoch 2600, Loss: -0.0493
Epoch 2700, Loss: -0.0493
Epoch 2800, Loss: -0.0493
Epoch 2900, Loss: -0.0493


In [15]:
predictions = predict(X, weights, bias)
print("Predicciones:")
print(predictions)

# Salidas reales para comparación
print("Salidas reales:")
print(y)

Predicciones:
[[3.01279126e-05 1.31053130e-05 8.02940916e-09]
 [1.01779578e-05 1.62507603e-10 1.07975407e-08]
 [4.07795799e-08 5.70703846e-07 5.06889133e-04]
 ...
 [8.90666795e-12 4.36076440e-08 5.67055932e-10]
 [4.79712893e-12 9.86434330e-01 1.99329277e-04]
 [2.23098467e-13 5.07623344e-19 5.92758148e-09]]
Salidas reales:
[[-0.66390084  1.34145628 -0.71274119]
 [ 1.50624903 -0.74545851 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 ...
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]]


In [16]:
mse = np.mean((y - predictions) ** 2)
mse

0.9684770401103653

Bonus 1

Para agregar  k capas ocultas con n neuronas cada una hay que hacer modificaciones en la funcion en la que se crean los weights y los bias, al igual que en la de forward propagation. Además, yo voy a usar otra funcion de activacion(ReLu) para las capas intermedias. 

In [17]:
#Creamos otra funcion de activacion para las capas ocultas que pide el bonus 
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [18]:
def bonus_initialize_weights(layer_sizes):
    # Creamos arrays vacios para almacenar los weights y bias
    weights = []
    biases = []

    # Crea valores random para todas las capas
    for i in range(len(layer_sizes) - 1):
        w = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01
        b = np.zeros((1, layer_sizes[i + 1]))
        
        # Agrega los pesos y sesgos a sus respectivas listas
        weights.append(w)
        biases.append(b)
    
    # Devuelve los arrays llenos
    return weights, biases


In [19]:
def bonus_forward_pass(X, weights, biases, layer_sizes):
    activations = [X]  # Valores que se utilizan como entrada para la siguiente capa

    # Bucle para recorrer las capas ocultas (excluye la última capa para oder utilizar otra funcion de activacion)
    for i in range(len(layer_sizes) - 2):  # -2 porque la última capa se trata por separado
        z = np.dot(activations[-1], weights[i]) + biases[i]
        a = relu(z)  # Aplicar función de activación ReLU a la salida de la capa actual
        activations.append(a)  # Añadir el valor calculado al array para que lo utilice la siguiente capa

    # Cálculo para la capa de salida
    z = np.dot(activations[-1], weights[-1]) + biases[-1]
    output = sigmoid(z)
    activations.append(output)

    # Devolver todos los valores calculados
    return activations


In [20]:
def bonus_backpropagation(X, y, activations, weights, biases, learning_rate):
    m = y.shape[0]  # Número de ejemplos
    L = len(weights)  # Número de capas
    
    # Se calcula el error y cuanto debe ajustarse de la capa de salida
    delta_z = activations[-1] - y 
    delta_weights = np.dot(activations[-2].T, delta_z) / m 
    delta_bias = np.sum(delta_z, axis=0, keepdims=True) / m 

    # Actualizar weights y bias de la capa de salida
    weights[-1] -= learning_rate * delta_weights
    biases[-1] -= learning_rate * delta_bias

    for l in range(L - 2, -1, -1):  # Desde la última capa oculta (L-2) hasta la primera capa (0)
    
        # Multiplicamos el gradiente de la capa siguiente (delta_z) por la transpuesta de los pesos de la capa actual (l+1).
        # Luego, multiplicamos por la derivada de la activación ReLU para ajustar el gradiente en la capa actual.
        delta_z = np.dot(delta_z, weights[l + 1].T) * relu_derivative(activations[l + 1])
        
        # Se hace el producto punto entre las activaciones transpuestas de la capa actual y delta_z.
        # El resultado se divide por el número de ejemplos en el lote (m) para obtener el promedio del gradiente.
        delta_weights = np.dot(activations[l].T, delta_z) / m
        
        # Sumamos todos los gradientes delta_z de la capa actual y promediamos dividiendo por el número de ejemplos (m).
        delta_bias = np.sum(delta_z, axis=0, keepdims=True) / m
        
        #Se actualizan los weights y bias de cada capa
        weights[l] -= learning_rate * delta_weights
        biases[l] -= learning_rate * delta_bias

    return weights, biases

In [21]:
def bonus_train(X, y, layer_sizes, epochs=1000, learning_rate=0.01):

    weights, biases = bonus_initialize_weights(layer_sizes)
    
    
    for epoch in range(epochs):
        activations = bonus_forward_pass(X, weights, biases, layer_sizes)
        
        loss = linear_error(y, activations[-1])

        weights, biases = bonus_backpropagation(X, y, activations, weights, biases, learning_rate)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    return weights, biases

In [22]:
def bonus_predict(X, weights, biases, layer_sizes):
    activations = bonus_forward_pass(X, weights, biases, layer_sizes)
    return activations[-1]

In [23]:
input_size = X.shape[1]
output_size = y.shape[1]

#Cantidad de neuronas por capa. Cada vez que se agrega un numero entre input y output size es una nueva capa, y el numero agregado representa la cantidad de neuronas que tendrá
layer_sizes = [input_size, 6,3,2, output_size]

epochs = 10000
learning_rate = 0.01
weights, biases = bonus_train(X, y, layer_sizes, epochs, learning_rate)

Epoch 0, Loss: -0.5000
Epoch 100, Loss: -0.3909
Epoch 200, Loss: -0.3116
Epoch 300, Loss: -0.2545
Epoch 400, Loss: -0.2129
Epoch 500, Loss: -0.1818
Epoch 600, Loss: -0.1579
Epoch 700, Loss: -0.1392
Epoch 800, Loss: -0.1242
Epoch 900, Loss: -0.1119
Epoch 1000, Loss: -0.1017
Epoch 1100, Loss: -0.0932
Epoch 1200, Loss: -0.0859
Epoch 1300, Loss: -0.0796
Epoch 1400, Loss: -0.0742
Epoch 1500, Loss: -0.0694
Epoch 1600, Loss: -0.0652
Epoch 1700, Loss: -0.0614
Epoch 1800, Loss: -0.0581
Epoch 1900, Loss: -0.0550
Epoch 2000, Loss: -0.0523
Epoch 2100, Loss: -0.0498
Epoch 2200, Loss: -0.0476
Epoch 2300, Loss: -0.0455
Epoch 2400, Loss: -0.0436
Epoch 2500, Loss: -0.0419
Epoch 2600, Loss: -0.0403
Epoch 2700, Loss: -0.0388
Epoch 2800, Loss: -0.0374
Epoch 2900, Loss: -0.0361
Epoch 3000, Loss: -0.0349
Epoch 3100, Loss: -0.0337
Epoch 3200, Loss: -0.0327
Epoch 3300, Loss: -0.0317
Epoch 3400, Loss: -0.0307
Epoch 3500, Loss: -0.0298
Epoch 3600, Loss: -0.0290
Epoch 3700, Loss: -0.0282
Epoch 3800, Loss: -0.027

In [24]:
bonus_predictions = bonus_predict(X, weights, biases, layer_sizes)
print("Predicciones:")
print(bonus_predictions)

print("Salidas reales:")
print(y)

Predicciones:
[[0.01026163 0.01026163 0.01026163]
 [0.01026163 0.01026163 0.01026163]
 [0.01026163 0.01026163 0.01026163]
 ...
 [0.01026163 0.01026163 0.01026163]
 [0.01026163 0.01026163 0.01026163]
 [0.01026163 0.01026163 0.01026163]]
Salidas reales:
[[-0.66390084  1.34145628 -0.71274119]
 [ 1.50624903 -0.74545851 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 ...
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]
 [-0.66390084  1.34145628 -0.71274119]]


Bonus 2


Lo que yo haría para que la salida sea mutualmente excluyente es cambiar la funcion de activación final. En vez de usar sigmoide, yo utilizaría SoftMax. Lo que esta funcion de activacion hace es que la suma de todas las columnas de la salida den 1. Al hacer esto, se puede interpretar como la probabilidad de que pertenezca a cada clase.

 En el dataset de tipos de conductor por ejemplo, haria que en vez de devolverme un rango entre 0 y 1 para cada posiblidad (Commuter, Long-Distance Traveler) me devuelva numeros que sumando todas las columnas den 1, haciendo que cada uno represente la posibilidad de que el conjunto original (X) perteneza a cada columna. Una vez tenga las "probabilidades", la que mayor chance tenga será a la que pertenece.